In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold


# test files
base_path = "input/"
test_xgb = pd.read_csv(base_path + 'xgb.csv')
test_lgb = pd.read_csv(base_path + 'lgb.csv')
test_dnn = pd.read_csv(base_path + 'dnn_predictions.csv')
test_up = pd.read_csv(base_path + 'test_submission.csv')
test_cat = pd.read_csv(base_path + 'catboost_submission.csv')
test_kin = pd.read_csv(base_path + 'test_uberKinetics.csv')
test_gp = pd.read_csv(base_path + 'test_gpari.csv')
test_bl = pd.read_csv('blend8.csv')


train=pd.read_csv(base_path + 'train.csv')
test=pd.read_csv(base_path + 'test.csv')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# train = pd.concat([train, 
#                    train_xgb[['target']].rename(columns = {'target' : 'xgb'}),
#                    train_lgb[['target']].rename(columns = {'target' : 'lgb'}),
#                    train_dnn[['target']].rename(columns = {'target' : 'dnn'}),
#                    train_up[['target']].rename(columns = {'target' : 'up'}),
#                    train_cat[['target']].rename(columns = {'target' : 'cat'}),
#                    train_kin[['target']].rename(columns = {'target' : 'kin'}),
#                    train_gp[['target']].rename(columns = {'target' : 'gp'})                   
#                   ], axis = 1)

test = pd.concat([test, 
                   test_xgb[['target']].rename(columns = {'target' : 'xgb'}),
                   test_lgb[['target']].rename(columns = {'target' : 'lgb'}),
                   test_dnn[['target']].rename(columns = {'target' : 'dnn'}),
                   test_up[['target']].rename(columns = {'target' : 'up'}),
                   test_cat[['target']].rename(columns = {'target' : 'cat'}),
                   test_kin[['target']].rename(columns = {'target' : 'kin'}),
                   test_gp[['target']].rename(columns = {'target' : 'gp'}),
                  ], axis = 1)


train_cols = ['xgb', 'lgb', 'dnn', 'up', 'cat', 'kin', 'gp']

In [11]:
for t in train_cols:
    max_ = train.append(test, ignore_index = True)[t].max()
    min_ = train.append(test, ignore_index = True)[t].min()
#     train[t] = (train[t] - min_) / (max_ - min_)
    test[t] = (test[t] - min_) / (max_ - min_)

for t in train_cols:
#     train[t + '_rank'] = train[t].rank()
    test[t + '_rank'] = test[t].rank()


test['target'] = (test['xgb_rank'] + test['lgb_rank'] + test['dnn_rank'] + test['up_rank'] + \
                 test['cat_rank'] + test['kin_rank'] + test['gp_rank']) / (7 * test.shape[0])

test[['id', 'target']].to_csv(base_path + 'rank_avg.csv.gz', index = False, compression = 'gzip') 